In [166]:
import zipfile
import os
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Get the current working directory (location of the notebook/script)
script_dir = os.getcwd()

print(f"Your script is located in: {script_dir}")
# Define directories
base_dir = "/Users/egeberk/Anaconda/Precipitation"
input_dir = os.path.join(base_dir, "Zipped")
output_dir = os.path.join(base_dir, "Data and Graphs")
extraction_dir = os.path.join(output_dir, "Extracted")

# Ensure directories exist
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(extraction_dir, exist_ok=True)

Your script is located in: /Users/egeberk/Anaconda/Precipitation


In [167]:
# Initialize a DataFrame to store results
Precipitation_Dataframe = pd.DataFrame()

# List all files in the input directory
if os.path.exists(input_dir):
    input_files = os.listdir(input_dir)
    print(f"Files in '{input_dir}':")
    for file in input_files:
        print(file)
else:
    print(f"The input directory '{input_dir}' does not exist.")

Files in '/Users/egeberk/Anaconda/Precipitation/Zipped':
Alice_Arm.zip
Hastings.zip
MountainRatz.zip
Juneau.zip
Hyder.zip
Chickamin.zip
Unuk.zip


In [168]:
def generate_and_save_graphs():
    """
    Generate and save graphs based on the Precipitation_Dataframe.
    """
    global Precipitation_Dataframe

    def get_Season(Month):
        if Month in [12, 1, 2]:
            return 'Winter'
        elif Month in [3, 4, 5]:
            return 'Spring'
        elif Month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Autumn'

    Precipitation_Dataframe['Season'] = Precipitation_Dataframe['Month'].apply(get_Season)

    def add_value_labels(ax, x_data, y_data, offset=1.0):
        """Adds value labels above points on the graph."""
        for x, y in zip(x_data, y_data):
            if not pd.isna(y):  # Skip NaN values
                ax.text(x, y + offset, f"{y:.1f}", ha='center', fontsize=10, color='black')  # Show values clearly

    # Define graph settings
    graph_types = [
        {
            "data": Precipitation_Dataframe.groupby(['Region', 'Year', 'Season'])['total_precipitation_intensity_mm_ha']
            .sum()
            .reset_index(),
            "title": "Seasonal Precipitation Intensity Trends",
            "column": "total_precipitation_intensity_mm_ha",  # Column name in the DataFrame
            "ylabel": "Total Precipitation Intensity (mm/ha)",  # User-friendly label
            "xlabel": "Year",
            "pivot": ('Year', 'Season', 'total_precipitation_intensity_mm_ha'),
            "filename": "Seasonal_precipitation_intensity_trends",
        },
        {
            "data": Precipitation_Dataframe.groupby(['Region', 'Year'])['total_precipitation_intensity_mm_ha']
            .sum()
            .reset_index(),
            "title": "Yearly Total Precipitation Intensity Trends",
            "column": "total_precipitation_intensity_mm_ha",  # Column name in the DataFrame
            "ylabel": "Total Precipitation Intensity (mm/ha)",  # User-friendly label
            "xlabel": "Year",
            "filename": "Yearly_total_precipitation_intensity_trends",
        },
        {
            "data": Precipitation_Dataframe.groupby(['Region', 'Month'])['mean_precipitation_intensity_mm_ha']
            .mean()
            .reset_index(),
            "title": "Monthly Mean Precipitation Intensity Trends",
            "column": "mean_precipitation_intensity_mm_ha",  # Column name in the DataFrame
            "ylabel": "Mean Precipitation Intensity (mm/ha)",  # User-friendly label
            "xlabel": "Month",
            "xticks": range(1, 13),
            "xticklabels": ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            "filename": "Monthly_mean_precipitation_intensity_trends",
        },
    ]

    # Generate graphs for each Region
    for graph in graph_types:
        data = graph["data"]
        Regions = data['Region'].unique()

        # Individual graphs
        for Region in Regions:
            Region_data = data[data['Region'] == Region]
            plt.figure(figsize=(14, 10))  # Larger figure size for better readability
            ax = plt.gca()
            if 'pivot' in graph:
                Region_pivot = Region_data.pivot(
                    index=graph["pivot"][0], columns=graph["pivot"][1], values=graph["pivot"][2]
                )
                for column in Region_pivot.columns:
                    ax.plot(Region_pivot.index, Region_pivot[column], marker='o', label=str(column))  # Add labels
                    add_value_labels(ax, Region_pivot.index, Region_pivot[column])  # Add numerical values
            else:
                x_data = Region_data[graph["xlabel"]]
                y_data = Region_data[graph["column"]]  # Use actual column name
                ax.plot(x_data, y_data, marker='o', label=Region)  # Add Region as label
                add_value_labels(ax, x_data, y_data)  # Add numerical values

            plt.title(f"{graph['title']} for {Region}")
            plt.ylabel(graph["ylabel"])  # Use user-friendly y-axis label
            plt.xlabel(graph["xlabel"])
            if "xticks" in graph:
                plt.xticks(graph["xticks"], graph.get("xticklabels", []))
            plt.grid()

            # Add legend only if labels are available
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                plt.legend()
            plt.savefig(os.path.join(output_dir, f"{Region}_{graph['filename']}.png"))
            plt.close()

        # Grid comparison graphs
        n_Regions = len(Regions)
        rows = (n_Regions + 1) // 2  # Two graphs per row

        # Consistent axis limits for all Regions
        y_min = data[graph["column"]].min()  # Use actual column name
        y_max = data[graph["column"]].max()  # Use actual column name

        plt.figure(figsize=(15, rows * 5))
        for i, Region in enumerate(Regions, start=1):
            Region_data = data[data['Region'] == Region]
            ax = plt.subplot(rows, 2, i)
            if 'pivot' in graph:
                Region_pivot = Region_data.pivot(
                    index=graph["pivot"][0], columns=graph["pivot"][1], values=graph["pivot"][2]
                )
                for column in Region_pivot.columns:
                    ax.plot(Region_pivot.index, Region_pivot[column], marker='o', label=str(column))  # Add labels
            else:
                x_data = Region_data[graph["xlabel"]]
                y_data = Region_data[graph["column"]]  # Use actual column name
                ax.plot(x_data, y_data, marker='o', label=Region)

            ax.set_title(Region)
            ax.set_xlabel(graph["xlabel"])
            ax.set_ylabel(graph["ylabel"])  # Use user-friendly y-axis label
            ax.set_ylim(y_min - 1, y_max + 1)  # Adjust y-limits for better visibility
            if "xticks" in graph:
                ax.set_xticks(graph["xticks"])
                ax.set_xticklabels(graph.get("xticklabels", []))
            ax.grid()

            # Add legend only if labels are available
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend()

        plt.suptitle(graph["title"], fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(os.path.join(output_dir, f"grid_{graph['filename']}.png"))
        plt.close()


In [169]:
def calculate_precipitation_intensities(Region_Area_file):
    """
    Add Area and precipitation intensity columns to the dataframe.

    Parameters:
        Region_Area_file (str): Path to the CSV file containing Region and Area data.
    """
    global Precipitation_Dataframe

    # Load the Region Area data with the correct delimiter
    Region_Area_df = pd.read_csv(Region_Area_file, delimiter=';')
    print("Region Area DataFrame loaded:")
    print(Region_Area_df.head())

    # Replace commas with dots and convert the Area column to numeric
    Region_Area_df['Area'] = Region_Area_df['Area'].str.replace(',', '.').astype(float)
    Region_Area_df['Area_ha'] = Region_Area_df['Area'] / 100000000  # Convert to hectares

    print("Converted Area to hectares:")
    print(Region_Area_df[['Region', 'Area_ha']].head())

    # Merge the dataframes on the Region column
    Precipitation_Dataframe = Precipitation_Dataframe.merge(
        Region_Area_df[['Region', 'Area_ha']], on='Region', how='left'
    )
    
    # Resolve duplicate columns
    if 'Area_ha_x' in Precipitation_Dataframe.columns or 'Area_ha_y' in Precipitation_Dataframe.columns:
        if 'Area_ha_y' in Precipitation_Dataframe.columns:
            Precipitation_Dataframe['Area_ha'] = Precipitation_Dataframe['Area_ha_y']
        elif 'Area_ha_x' in Precipitation_Dataframe.columns:
            Precipitation_Dataframe['Area_ha'] = Precipitation_Dataframe['Area_ha_x']
        Precipitation_Dataframe.drop(columns=['Area_ha_x', 'Area_ha_y'], inplace=True, errors='ignore')
        print("Resolved duplicate columns, retaining 'Area_ha'.")


    print("After merging Area data:")
    print(Precipitation_Dataframe.head())

    # Check for missing Area_ha values
    if Precipitation_Dataframe['Area_ha'].isnull().any():
        print("Warning: Missing Area_ha values detected.")
        Precipitation_Dataframe['Area_ha'] = Precipitation_Dataframe['Area_ha'].fillna(0)  # Default to 0 if needed

    # Calculate precipitation intensities using vectorized operations
    for col in ['total_precipitation_mm', 'mean_precipitation_mm', 'min_precipitation_mm', 'max_precipitation_mm']:
        intensity_col = col.replace("_mm", "_intensity_mm_ha")
        Precipitation_Dataframe[intensity_col] = Precipitation_Dataframe[col] / Precipitation_Dataframe['Area_ha']


In [170]:
def generate_and_save_graphs():
    """
    Generate and save graphs based on the Precipitation_Dataframe.
    """
    global Precipitation_Dataframe

    def get_Season(Month):
        if Month in [12, 1, 2]:
            return 'Winter'
        elif Month in [3, 4, 5]:
            return 'Spring'
        elif Month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Autumn'

    Precipitation_Dataframe['Season'] = Precipitation_Dataframe['Month'].apply(get_Season)

    def add_value_labels(ax, x_data, y_data, offset=1.0):
        """Adds value labels above points on the graph."""
        for x, y in zip(x_data, y_data):
            if not pd.isna(y):  # Skip NaN values
                ax.text(x, y + offset, f"{y:.1f}", ha='center', fontsize=10, color='black')  # Show values clearly

    # Define graph settings
    graph_types = [
        {
            "data": Precipitation_Dataframe.groupby(['Region', 'Year', 'Season'])['total_precipitation_intensity_mm_ha']
            .sum()
            .reset_index(),
            "title": "Seasonal Precipitation Intensity Trends",
            "column": "total_precipitation_intensity_mm_ha",  # Column name in the DataFrame
            "ylabel": "Total Precipitation Intensity (mm/ha)",  # User-friendly label
            "xlabel": "Year",
            "pivot": ('Year', 'Season', 'total_precipitation_intensity_mm_ha'),
            "filename": "Seasonal_precipitation_intensity_trends",
        },
        {
            "data": Precipitation_Dataframe.groupby(['Region', 'Year'])['total_precipitation_intensity_mm_ha']
            .sum()
            .reset_index(),
            "title": "Yearly Total Precipitation Intensity Trends",
            "column": "total_precipitation_intensity_mm_ha",  # Column name in the DataFrame
            "ylabel": "Total Precipitation Intensity (mm/ha)",  # User-friendly label
            "xlabel": "Year",
            "filename": "Yearly_total_precipitation_intensity_trends",
        },
        {
            "data": Precipitation_Dataframe.groupby(['Region', 'Month'])['mean_precipitation_intensity_mm_ha']
            .mean()
            .reset_index(),
            "title": "Monthly Mean Precipitation Intensity Trends",
            "column": "mean_precipitation_intensity_mm_ha",  # Column name in the DataFrame
            "ylabel": "Mean Precipitation Intensity (mm/ha)",  # User-friendly label
            "xlabel": "Month",
            "xticks": range(1, 13),
            "xticklabels": ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            "filename": "Monthly_mean_precipitation_intensity_trends",
        },
    ]

    # Generate graphs for each Region
    for graph in graph_types:
        data = graph["data"]
        Regions = data['Region'].unique()

        # Individual graphs
        for Region in Regions:
            Region_data = data[data['Region'] == Region]
            plt.figure(figsize=(14, 10))  # Larger figure size for better readability
            ax = plt.gca()
            if 'pivot' in graph:
                Region_pivot = Region_data.pivot(
                    index=graph["pivot"][0], columns=graph["pivot"][1], values=graph["pivot"][2]
                )
                for column in Region_pivot.columns:
                    ax.plot(Region_pivot.index, Region_pivot[column], marker='o', label=str(column))  # Add labels
                    add_value_labels(ax, Region_pivot.index, Region_pivot[column])  # Add numerical values
            else:
                x_data = Region_data[graph["xlabel"]]
                y_data = Region_data[graph["column"]]  # Use actual column name
                ax.plot(x_data, y_data, marker='o', label=Region)  # Add Region as label
                add_value_labels(ax, x_data, y_data)  # Add numerical values

            plt.title(f"{graph['title']} for {Region}")
            plt.ylabel(graph["ylabel"])  # Use user-friendly y-axis label
            plt.xlabel(graph["xlabel"])
            if "xticks" in graph:
                plt.xticks(graph["xticks"], graph.get("xticklabels", []))
            plt.grid()

            # Add legend only if labels are available
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                plt.legend()
            plt.savefig(os.path.join(output_dir, f"{Region}_{graph['filename']}.png"))
            plt.close()

        # Grid comparison graphs
        n_Regions = len(Regions)
        rows = (n_Regions + 1) // 2  # Two graphs per row

        # Consistent axis limits for all Regions
        y_min = data[graph["column"]].min()  # Use actual column name
        y_max = data[graph["column"]].max()  # Use actual column name

        plt.figure(figsize=(15, rows * 5))
        for i, Region in enumerate(Regions, start=1):
            Region_data = data[data['Region'] == Region]
            ax = plt.subplot(rows, 2, i)
            if 'pivot' in graph:
                Region_pivot = Region_data.pivot(
                    index=graph["pivot"][0], columns=graph["pivot"][1], values=graph["pivot"][2]
                )
                for column in Region_pivot.columns:
                    ax.plot(Region_pivot.index, Region_pivot[column], marker='o', label=str(column))  # Add labels
            else:
                x_data = Region_data[graph["xlabel"]]
                y_data = Region_data[graph["column"]]  # Use actual column name
                ax.plot(x_data, y_data, marker='o', label=Region)

            ax.set_title(Region)
            ax.set_xlabel(graph["xlabel"])
            ax.set_ylabel(graph["ylabel"])  # Use user-friendly y-axis label
            ax.set_ylim(y_min - 1, y_max + 1)  # Adjust y-limits for better visibility
            if "xticks" in graph:
                ax.set_xticks(graph["xticks"])
                ax.set_xticklabels(graph.get("xticklabels", []))
            ax.grid()

            # Add legend only if labels are available
            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend()

        plt.suptitle(graph["title"], fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(os.path.join(output_dir, f"grid_{graph['filename']}.png"))
        plt.close()


In [171]:
print(Precipitation_Dataframe.columns)


RangeIndex(start=0, stop=0, step=1)


In [172]:
# example usage
Region_Area_file = "/users/egeberk/anaconda/Precipitation/Watershed Area.csv"
if os.path.exists(input_dir):
    for zip_file in os.listdir(input_dir):
        if zip_file.endswith(".zip"):
            zip_path = os.path.join(input_dir, zip_file)
            Region_name = os.path.splitext(zip_file)[0]
            process_uploaded_zip(zip_path, Region_name)

calculate_precipitation_intensities(Region_Area_file)
generate_and_save_graphs()

# save the Precipitation_Dataframe to the folder
Precipitation_Dataframe.to_csv(os.path.join(output_dir, "PrecipitationDataFrame.csv"), index=False)
Precipitation_Dataframe.to_excel(os.path.join(output_dir, "PrecipitationDataframe.xlsx"), index=False)

print("Precipitation_Dataframe has been saved as csv and excel.")

extracted files: ['reprojected_pr_total_mm_cru_ts405_historical_12_1993.tif', 'reprojected_pr_total_mm_cru_ts405_historical_09_2008.tif', 'reprojected_pr_total_mm_cru_ts405_historical_08_2008.tif', 'reprojected_pr_total_mm_cru_ts405_historical_11_2013.tif', 'reprojected_pr_total_mm_cru_ts405_historical_10_2013.tif', 'clipped_pr_total_mm_cru_ts405_historical_09_2008.tif', 'clipped_pr_total_mm_cru_ts405_historical_08_2008.tif', 'clipped_pr_total_mm_cru_ts405_historical_12_1993.tif', 'clipped_pr_total_mm_cru_ts405_historical_11_2013.tif', 'clipped_pr_total_mm_cru_ts405_historical_10_2013.tif', 'clipped_pr_total_mm_cru_ts405_historical_04_1983.tif', 'clipped_pr_total_mm_cru_ts405_historical_05_1983.tif', 'clipped_pr_total_mm_cru_ts405_historical_06_2003.tif', 'clipped_pr_total_mm_cru_ts405_historical_07_2003.tif', 'reprojected_pr_total_mm_cru_ts405_historical_01_1988.tif', 'reprojected_pr_total_mm_cru_ts405_historical_03_2008.tif', 'reprojected_pr_total_mm_cru_ts405_historical_02_2008.tif'